## Содержание
1.  [Подготовка](#подготовка)
2.  [Обучение](#обучение)
3.  [Выводы](#выводы)
4.  [Чек-лист проверки](#чек)

## Проект для «Викишоп» c Bert

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75.

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели.
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

In [1]:
from google.colab import output
output.eval_js('function ClickConnect(){console.log("Preventing disconnect");document.querySelector("colab-toolbar-button#connect").click()}setInterval(ClickConnect,60000)')

## <a name='подготовка'></a> Подготовка

**Загрузка библиотек**

In [2]:
!pip install catboost -q

In [3]:
import torch
import numpy as np
import pandas as pd
import transformers
from tqdm import notebook
from nltk.corpus import stopwords
from sklearn.metrics import f1_score
from catboost import CatBoostClassifier
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

**Загрузка данных**

In [4]:
try:
  df = pd.read_csv('/content/drive/MyDrive/Yandex_Practicum/toxic_comments.csv')
except:
  df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')

df.info()

In [5]:
# Проверка распределения классов
class_distribution = df['toxic'].value_counts(normalize=True)
class_distribution

Сильный дисбаланс классов. Используем метод балансировки при обучении модели LogisticRegression

In [6]:
# Оставим только часть, так как не хватает вычислительной мощности для всей выборки
df_sample = df.sample(1000).reset_index(drop=True)

In [7]:
# Удаление ненужных символов
df_sample['text'] = df_sample['text'].replace(to_replace='[^\w\s]', value='', regex=True)

# Загрузка предобученного токенизатора и модели
model_name = 'unitary/toxic-bert'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# подготовим устройство для обучения
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

# Токенизация текстов
tokenized = df_sample['text'].apply(
    lambda x: tokenizer.encode(x, add_special_tokens=True, truncation=True, max_length=512))

# Определение максимальной длины среди всех токенизированных текстов
max_len = max(len(seq) for seq in tokenized)

# Приведение всех последовательностей к одной длине (max_len) путем добавления паддинга (нулями)
padded = np.array([seq + [0] * (max_len - len(seq)) for seq in tokenized])

# Создание маски внимания: 1 для реальных токенов, 0 для токенов-паддинга
attention_mask = np.where(padded != 0, 1, 0)

# Установка размера батча
batch_size = 100
embeddings = []

# помещаем модель на GPU
model.to(device)

# Отключение автоматического вычисления градиентов (не требуется при инференсе)
with torch.no_grad():
    # Итерация по данным с шагом, равным размеру батча
    for i in notebook.tqdm(range(0, len(padded), batch_size)):
        # Формирование батча данных и маски внимания
        batch = torch.LongTensor(padded[i:i + batch_size])
        attention_mask_batch = torch.LongTensor(attention_mask[i:i + batch_size])
        # Получение выходов модели
        outputs = model(batch.to(device), # помещаем данные на GPU
                        attention_mask=attention_mask_batch.to(device))
        # Извлечение эмбеддингов CLS-токена и преобразование их в numpy-массив
        embeddings.append(outputs.last_hidden_state[:, 0, :].cpu().numpy())

# Объединение всех эмбеддингов в один массив
features = np.concatenate(embeddings)

# Отображение формы полученного массива эмбеддингов
display(features.shape)

In [8]:
X = features
y = df_sample['toxic']

# Разделение на обучающую и тестовую выборки
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## <a name='обучение'></a> Обучение

**Модель LinearRegression**

In [9]:
# Создание экземпляра LogisticRegression
lr = LogisticRegression(random_state=42, class_weight='balanced')

# Определение сетки гиперпараметров для поиска
param_grid_lr = {'C': [0.01, 0.1, 1, 10],
                 'max_iter': [500, 1000]}

# Настройка GridSearchCV
grid_lr = GridSearchCV(lr, param_grid_lr, scoring='f1', cv=5)

# Поиск лучших гиперпараметров
grid_lr.fit(X_train_val, y_train_val)

# модель с лучшими параметрами
best_lr = grid_lr.best_estimator_
print(f'Лучшие параметры для LogisticRegression: {grid_lr.best_params_}')
print(f'F1 по кросс-валидации: {grid_lr.best_score_}')

**Модель CatBoost**

In [10]:
# Создание экземпляра CatBoostClassifier
cb = CatBoostClassifier(random_state=42, verbose=0, task_type='GPU', devices='0')

# Определение сетки гиперпараметров для поиска
param_grid_cb = {
    'iterations': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'depth': [4, 6, 8]
}

# Настройка GridSearchCV
grid_cb = GridSearchCV(cb, param_grid_cb, scoring='f1', cv=5)

# Поиск лучших гиперпараметров
grid_cb.fit(X_train_val, y_train_val)

# Модель с лучшими параметрами
best_cb = grid_cb.best_estimator_

print(f'Лучшие параметры для CatBoost: {grid_cb.best_params_}')
print(f'F1 по кросс-валидации: {grid_cb.best_score_}')

In [11]:
# --- Выбираем лучшую модель ---
if grid_lr.best_score_ > grid_cb.best_score_:
    final_model = best_lr
    model_name = "LogisticRegression"
else:
    final_model = best_cb
    model_name = "CatBoost"

# Тестируем лучшую модель
y_pred_final = final_model.predict(X_test)
f1_final = f1_score(y_test, y_pred_final)
print(f'Лучшая модель: {model_name}, F1 на тесте: {f1_final}')

## <a name='выводы'></a> Выводы

В рамках проекта для интернет-магазина «Викишоп» была поставлена задача разработать модель для автоматического выявления токсичных комментариев, чтобы направлять их на модерацию. Цель заключалась в обучении модели, способной классифицировать комментарии на позитивные и негативные, с достижением значения метрики F1 не ниже 0.75.

**Ход работы:**

*Предобработка данных:*

Проведён анализ распределения классов, выявивший значительный дисбаланс: около 10% комментариев были токсичными, остальные — нет.
Для ускорения вычислений была выбрана случайная подвыборка из 800 комментариев.
Из текста комментариев удалены ненужные символы с помощью регулярных выражений.

В ходе работы были исследованы две модели классификации: логистическая регрессия и CatBoost. Обе модели обучались на эмбеддингах комментариев, полученных с использованием предобученной модели 'unitary/toxic-bert'.  Для учета дисбаланса классов в обучающих моделях был применен параметр class_weight='balanced' для логистической регрессии и соответствующие механизмы обработки дисбаланса в CatBoost (реализованные по умолчанию). Для поиска оптимальных гиперпараметров и среднего значения метрики использовалась кросс-валидация GridSearchCV.

*Результаты оценки качества на кросс-валидации показали следующее:*

- Логистическая регрессия с параметрами {'C': 0.1, 'max_iter': 500}: Достигнуто значение метрики F1 на уровне 0.909.
- CatBoost с параметрами {'depth': 6, 'iterations': 150, 'learning_rate': 0.01}: Продемонстрировано значительно более высокое значение метрики F1, равное 0.93.

*Результаты оценки качества на тестовой выборке показали следующее:*
- CatBoost продемонстрировало значение метрики F1 на уровне 0.92, что незначительно отличается от значения метрики на тренировчно-валидационной выборке.

**Вывод:**

Таким образом, обе разработанные модели успешно справились с задачей классификации токсичных комментариев и превысили установленный целевой показатель F1 в 0.75.  Однако, модель CatBoost продемонстрировала незначительно более высокое качество классификации, достигнув F1-score 0.93 на валидации и 0.92 на тесте., что указывает на ее эффективность в решении поставленной задачи.